In [ ]:
import PIL
from PIL import Image, ImageDraw
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
import numpy as np
import os
import math
import random
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point
import itertools

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/yolo_custom_model_Training/

/content/drive/My Drive/yolo_custom_model_Training


In [ ]:
def escolher_colonia():
  i_colonia = np.random.randint(1,21)
  tamanho_colonia = np.random.randint(20,51)
  colonia = Image.open(f'colonias/{i_colonia}.png').resize((tamanho_colonia,tamanho_colonia))
  return colonia

def escolher_fundo():
  i_fundo = np.random.randint(1,16+1)
  fundo = Image.open(f'fundos/fundo{i_fundo}.png')
  return fundo

In [ ]:
def distance(a, b):
    return Point(a).distance(Point(b))

class Square(object):
    def __init__(self,fundo_copia):
        self.colonia = escolher_colonia()
        self.fundo_size= fundo_copia.size
        self.espacosx = [(877,1047),(1217,1677),(1049,1831),(1199,1667),(1831,2000)]
        self.espacosy = [(625,1439), (368,476) ,(480,1573) ,(1575,1656),(632,1414)]
        self.i_espaço = np.random.choice([0,1,2,3,4],p= [0.125,0.125,0.5,0.125,0.125]) #Esquerda, cima, centro, baixo, direita
        self.x0, self.y0 = np.random.randint(low=self.espacosx[self.i_espaço][0],high=self.espacosx[self.i_espaço][1]-self.colonia.size[0]),\
                                             np.random.randint(low=self.espacosy[self.i_espaço][0],high=self.espacosy[self.i_espaço][1]-self.colonia.size[1])
        self.x1 = self.x0 + self.colonia.size[0]
        self.x2 = self.x1 
        self.x3 = self.x0
        self.y1 = self.y0
        self.y2 = self.y1 + self.colonia.size[1]
        self.y3 = self.y2                                       # Esquema do quadrado:  
        self.corners = ((self.x0, self.y0), (self.x1, self.y1), #                 [0]       [1]         [2]           [3]
                        (self.x2, self.y2), (self.x3, self.y3)) #     Pontos:   up left   up right   down right    down left
    @property
    def center(self):
        """(x, y) centro do polígono."""
        return Polygon(self.corners).centroid.coords[0]

    @property
    def half_diag(self):
        """Distância do centro ao canto"""
        p0, p1, p2, p3 = self.corners
        return 0.5 * distance(p0, p1) * math.sqrt(2)

    @property
    def yolo_coord(self):
        """As coordenadas da colonia no padrão Yolo normalized[xcenter,ycenter,width,height]"""
        xcenter = self.center[0]/self.fundo_size[0]
        ycenter = self.center[1]/self.fundo_size[1]
        width   = self.colonia.size[0] /self.fundo_size[0]
        height  = self.colonia.size[1] /self.fundo_size[1]
        return [xcenter,ycenter,width,height]
    
def test_overlap(square1, square2):

  # Distância entre dois centros
  ctc = distance(square1.center, square2.center)
  # Soma entre as half diagonals
  halfdiags = square1.half_diag + square2.half_diag
  res = ctc < halfdiags
  return res

In [ ]:
class Squares(object):
    def __init__(self):
        self.squares = []

    def add_square(self,fundo_copia):
        new_square = Square(fundo_copia)
        if not self.squares:
            self.squares.append(new_square)
        else:
            while True:
            # Testa se o novo quadrado faz overlap com o existente
                res = [test_overlap(square, new_square) for square in self.squares]
                if any(res):
                    # No caso de sobrepor algum
                    new_square = Square(fundo_copia)
                else:
                    # Adicionando à coleção
                    self.squares.append(new_square)
                    break

    def plot_squares(self,fundo):
        text_img = Image.new('RGBA', fundo.size, (0, 0, 0, 0))
        text_img.paste(fundo, (0,0))
        texto = []
        for square in self.squares:
            (x0, y0), (x1, y1), (x2, y2), (x3, y3) = square.corners
            texto.append(square.yolo_coord)
            text_img.paste(square.colonia, (x0,y0), mask=square.colonia)
        return text_img,texto

In [ ]:
%matplotlib inline
def colar_embaralhado (quantidade,indice,save= True,name=None):
  fundo = escolher_fundo()
  fundo_copia = fundo.copy()
  quadrados= Squares()
  for _ in itertools.repeat(None, quantidade):
    quadrados.add_square(fundo_copia)
  if save:
    img,coord = quadrados.plot_squares(fundo_copia)
    img.save(f'colonia_check/{indice}%{quantidade}%.png')

In [ ]:
#Digite quantas imagens quer logo abaixo
for indice,i in enumerate(np.arange(1,201)):
  name= str(i)
  quantidade = np.random.randint(5,41)
  colar_embaralhado(quantidade,indice,save= True,name=name)